# PSyKE's demo for regression tasks

Some imports.

In [1]:
from psyke import Extractor, Clustering, EvaluableModel
from psyke.tuning.pedro import PEDRO
from psyke.tuning import Objective
from psyke.tuning.crash import CRASH
from sklearn.tree import DecisionTreeRegressor
from psyke.utils.logic import pretty_theory
from psyke.utils.metrics import mae, mse, r2
from sklearn.model_selection import train_test_split
from psyke.utils import Target
import pandas as pd

Import a dataset.

In [2]:
dataset = pd.read_csv("../test/resources/datasets/df.csv")
dataset = dataset[["X", "Y", "Z0"]].dropna()
#dataset = pd.read_csv("../test/resources/datasets/CCPP.csv", sep=";", decimal=",")
#dataset

Split between train and test set in a reproducible way.

In [3]:
train, test = train_test_split(dataset, test_size=0.85, random_state=10)

In [4]:
#from psyke.tuning.orchid import OrCHiD

#orchid = OrCHiD(dataframe=train, algorithm=OrCHiD.Algorithm.CREAM, output=Target.REGRESSION,
#                max_mae_increase=1.2, min_rule_decrease=0.9, readability_tradeoff=0.1, patience=5, max_depth=3)
#orchid.search()
#(_, _, depth, threshold) = orchid.get_best()[0]

We use as predictor a KNN and we train it.

In [5]:
#predictor = KNeighborsRegressor(n_neighbors=3).fit(train.iloc[:, :-1], train.iloc[:, -1])
predictor = DecisionTreeRegressor().fit(train.iloc[:, :-1], train.iloc[:, -1])
#predictor = LinearRegression().fit(train.iloc[:, :-1], train.iloc[:, -1])

predicted = predictor.predict(test.iloc[:, :-1]).flatten()
true = test.iloc[:, -1]

print(f'MAE = {mae(true, predicted):.2f}')
print(f'MSE = {mse(true, predicted):.2f}')
print(f'R2 = {r2(true, predicted):.2f}')

MAE = 0.00
MSE = 0.00
R2 = 1.00


We define a function to print the extractors' evaluation

In [6]:
def print_scores(scores):
    print(f'MAE = {scores[EvaluableModel.RegressionScore.MAE][0]:.2f} (data), '
          f'{scores[EvaluableModel.RegressionScore.MAE][1]:.2f} (BB)\n'
          f'MSE = {scores[EvaluableModel.RegressionScore.MSE][0]:.2f} (data), '
          f'{scores[EvaluableModel.RegressionScore.MSE][1]:.2f} (BB)\n'
          f'R2 = {scores[EvaluableModel.RegressionScore.R2][0]:.2f} (data), '
          f'{scores[EvaluableModel.RegressionScore.R2][1]:.2f} (BB)')

def get_scores(extractor, test, predictor):
    return extractor.score(test, predictor, True, True, task=EvaluableModel.Task.REGRESSION,
                           scoring_function=[EvaluableModel.RegressionScore.MAE, EvaluableModel.RegressionScore.MSE,
                            EvaluableModel.RegressionScore.R2])

We create several extractors that use ITER, GridEx and GridREx algorithms to extract prolog rules from the predictor.

In [7]:
cosmik = Extractor.cosmik(predictor, max_components=4, k=150, patience=15, close_to_center=True)
theory_from_cosmik = cosmik.extract(train)
scores, completeness = get_scores(cosmik, test, predictor)
print(f'COSMiK performance ({cosmik.n_rules} rules with {completeness * 100:.2f}% coverage):')
print_scores(scores)
print('\nCOSMiK extracted rules:\n\n' + pretty_theory(theory_from_cosmik))

COSMiK performance (3 rules with 90.03% coverage):
MAE = 0.11 (data), 0.11 (BB)
MSE = 0.05 (data), 0.05 (BB)
R2 = 0.92 (data), 0.92 (BB)

COSMiK extracted rules:

'Z0'(X, Y, 1.0) :-
    X in [0.0, 0.48], Y in [0.0, 0.48].
'Z0'(X, Y, -0.98) :-
    X in [0.65, 1.0], Y in [0.0, 0.46].
'Z0'(X, Y, -0.13) :-
    X in [0.05, 0.99], Y in [0.59, 1.0].


In [8]:
cosmik = Extractor.cosmik(predictor, max_components=4, k=150, patience=15, close_to_center=True, output=Target.REGRESSION)
theory_from_cosmik = cosmik.extract(train)
scores, completeness = get_scores(cosmik, test, predictor)
print(f'COSMiK performance ({cosmik.n_rules} rules with {completeness * 100:.2f}% coverage):')
print_scores(scores)
print('\nCOSMiK extracted rules:\n\n' + pretty_theory(theory_from_cosmik))

COSMiK performance (4 rules with 89.92% coverage):
MAE = 0.08 (data), 0.08 (BB)
MSE = 0.03 (data), 0.03 (BB)
R2 = 0.96 (data), 0.96 (BB)

COSMiK extracted rules:

'Z0'(X, Y, Z0) :-
    X in [0.0, 0.48], Y in [0.0, 0.48], Z0 is 1.0.
'Z0'(X, Y, Z0) :-
    X in [0.65, 1.0], Y in [0.0, 0.44], Z0 is -1.0.
'Z0'(X, Y, Z0) :-
    X in [0.94, 1.0], Y in [0.79, 1.0], Z0 is -1.0.
'Z0'(X, Y, Z0) :-
    X in [0.05, 0.98], Y in [0.57, 0.99], Z0 is 0.53 - 0.82 * X - 0.28 * Y.


In [10]:
pedro = PEDRO(predictor, train, max_mae_increase=1.2, min_rule_decrease=0.9, readability_tradeoff=0.1,
              max_depth=1, patience=1, algorithm=PEDRO.Algorithm.GRIDEX, objective=Objective.MODEL)
pedro.search()
(_, _, threshold, grid) = pedro.get_best()[0]

gridEx = Extractor.gridex(predictor, grid, threshold=threshold)
theory_from_gridEx = gridEx.extract(train)
scores, completeness = get_scores(gridEx, test, predictor)
print(f'GridEx performance ({gridEx.n_rules} rules with {completeness * 100:.2f}% coverage):')
print_scores(scores)
print('\nGridEx extracted rules:\n\n' + pretty_theory(theory_from_gridEx))

Algorithm.GRIDEX. Grid (1). Fixed (2). Threshold = 0.00. MAE = 4.04, 4 rules
Algorithm.GRIDEX. Grid (1). Fixed (2). Threshold = 0.00. MAE = 4.04, 4 rules

Algorithm.GRIDEX. Grid (1). Fixed (3). Threshold = 0.00. MAE = 2.23, 9 rules
Algorithm.GRIDEX. Grid (1). Fixed (3). Threshold = 0.00. MAE = 2.23, 9 rules

Algorithm.GRIDEX. Grid (1). Adaptive ([(0.99, 2)]). Threshold = 0.00. MAE = 4.06, 2 rules
Algorithm.GRIDEX. Grid (1). Adaptive ([(0.99, 2)]). Threshold = 0.00. MAE = 4.06, 2 rules

Algorithm.GRIDEX. Grid (1). Adaptive ([(0.99, 3)]). Threshold = 0.00. MAE = 3.79, 3 rules
Algorithm.GRIDEX. Grid (1). Adaptive ([(0.99, 3)]). Threshold = 0.00. MAE = 3.79, 3 rules

Algorithm.GRIDEX. Grid (1). Adaptive ([(0.99, 5)]). Threshold = 0.00. MAE = 2.75, 5 rules
Algorithm.GRIDEX. Grid (1). Adaptive ([(0.99, 5)]). Threshold = 0.00. MAE = 2.75, 5 rules

Algorithm.GRIDEX. Grid (1). Adaptive ([(0.99, 10)]). Threshold = 0.00. MAE = 2.78, 10 rules
Algorithm.GRIDEX. Grid (1). Adaptive ([(0.99, 10)]). Th

In [8]:
orbit = Extractor.orbit(predictor, depth=3, error_threshold=1.0, gauss_components=2, output=Target.REGRESSION)
theory_from_orbit = orbit.extract(train)
scores, completeness = get_scores(orbit, test, predictor)
print(f'ORBIt performance ({orbit.n_rules} rules with {completeness * 100:.2f}% coverage):')
print_scores(scores)
print('\nORBIt extracted rules:\n\n' + pretty_theory(theory_from_orbit))

ORBIt performance (2 rules with 100.00% coverage):
MAE = 4.15 (data), 4.15 (BB)
MSE = 28.12 (data), 28.10 (BB)
R2 = -0.13 (data), -0.13 (BB)

ORBIt extracted rules:

'Z4'(X, Y, 15.42) :-
    0.02 * X - 0.03 * Y =< -0.01, -0.02 * X + 0.03 * Y =< 0.01, 1.0 * X + 0.0 * Y =< 0.46, -1.0 * X + 0.0 * Y =< -0.16.
'Z4'(X, Y, 5.053691).


In [14]:
creepy = Extractor.creepy(predictor, depth=3, error_threshold=0.02, output=Target.REGRESSION,
                          clustering=Clustering.exact)
theory_from_creepy = creepy.extract(train)
scores, completeness = get_scores(creepy, test, predictor)
print(f'CReEPy performance ({creepy.n_rules} rules with {completeness * 100:.2f}% coverage):')
print_scores(scores)
print('\nCReEPy extracted rules (ExACT):\n\n' + pretty_theory(theory_from_creepy))

CReEPy performance (3 rules with 100.00% coverage):
MAE = 0.68 (data), 0.71 (BB)
MSE = 2.05 (data), 2.06 (BB)
R2 = 0.92 (data), 0.92 (BB)

CReEPy extracted rules (ExACT):

'Z4'(X, Y, Z4) :-
    X in [0.15, 0.84], Y in [-0.00, 0.59], Z4 is 7.49 - 7.63 * X + 12.05 * Y.
'Z4'(X, Y, Z4) :-
    X in [0.15, 0.84], Y in [-0.00, 0.84], Z4 is 9.0 - 12.0 * X + 15.0 * Y.
'Z4'(X, Y, Z4) :-
    Y in [-0.00, 1.00], Z4 is 2.0 + 4.0 * X - 3.0 * Y.


In [20]:
creepy = Extractor.creepy(predictor, depth=2, error_threshold=0.02, output=Target.REGRESSION,
                          clustering=Clustering.cream)
theory_from_creepy = creepy.extract(train)
scores, completeness = get_scores(creepy, test, predictor)
print(f'CReEPy performance ({creepy.n_rules} rules with {completeness * 100:.2f}% coverage):')
print_scores(scores)
print('\nCReEPy extracted rules (CREAM):\n\n' + pretty_theory(theory_from_creepy))

CReEPy performance (4 rules with 100.00% coverage):
MAE = 3.37 (data), 3.47 (BB)
MSE = 18.62 (data), 19.19 (BB)
R2 = 0.94 (data), 0.93 (BB)

CReEPy extracted rules (CREAM):

'PE'(AP, AT, RH, V, PE) :-
    AT in [6.21, 32.45], V in [34.02, 50.16], AP in [997.90, 1026.41], RH in [35.63, 100.10], PE is 502.53 - 2.16 * AP - 0.26 * AT + 0.01 * RH - 0.11 * V.
'PE'(AP, AT, RH, V, PE) :-
    AT in [6.21, 35.77], V in [25.35, 81.56], AP in [997.84, 1026.45], RH in [25.55, 100.12], PE is 234.73 - 1.42 * AP - 0.29 * AT + 0.26 * RH - 0.12 * V.
'PE'(AP, AT, RH, V, PE) :-
    AT in [3.30, 14.60], V in [34.68, 44.47], AP in [1011.31, 1033.25], RH in [58.98, 98.68], PE is 720.26 - 2.20 * AP - 0.47 * AT - 0.18 * RH - 0.22 * V.
'PE'(AP, AT, RH, V, PE) :-
    V in [25.35, 81.56], AP in [992.88, 1033.25], RH in [25.55, 100.15], PE is 579.01 - 2.05 * AP - 0.60 * AT - 0.05 * RH + 0.00 * V.


In [21]:
crash = CRASH(predictor, train, max_depth=3, patience=1, readability_tradeoff=.5,
              algorithm=CRASH.Algorithm.ExACT, output=Target.REGRESSION)
crash.search()
(_, _, depth, threshold) = crash.get_best()[0]

creepy = Extractor.creepy(predictor, depth=depth, error_threshold=threshold, output=Target.REGRESSION,
                          clustering=Clustering.exact)
theory_from_creepy = creepy.extract(train)
scores, completeness = get_scores(creepy, test, predictor)
print(f'CReEPy performance ({creepy.n_rules} rules with {completeness * 100:.2f}% coverage):')
print_scores(scores)
print('\nCReEPy extracted rules:\n\n' + pretty_theory(theory_from_creepy))

Algorithm.ExACT. Depth: 1. Threshold = 0.00. MAE = 3.52, 2 rules
Algorithm.ExACT. Depth: 1. Threshold = 0.00. MAE = 3.54, 2 rules

Algorithm.ExACT. Depth: 2. Threshold = 0.00. MAE = 3.52, 3 rules
Algorithm.ExACT. Depth: 2. Threshold = 0.00. MAE = 3.48, 3 rules
Algorithm.ExACT. Depth: 2. Threshold = 0.00. MAE = 3.50, 3 rules

Algorithm.ExACT. Depth: 3. Threshold = 0.00. MAE = 3.44, 4 rules
Algorithm.ExACT. Depth: 3. Threshold = 0.00. MAE = 3.41, 4 rules
Algorithm.ExACT. Depth: 3. Threshold = 0.00. MAE = 3.52, 4 rules

**********************
Best Algorithm.ExACT
**********************
MAE = 3.41, 4 rules
Threshold = 0.00
Depth = 3

**********************
Best   MAE  
**********************
MAE = 3.41, 4 rules
Threshold = 0.00
Depth = 3

**********************
Best N rules
**********************
MAE = 3.54, 2 rules
Threshold = 0.00
Depth = 1

CReEPy performance (4 rules with 100.00% coverage):
MAE = 3.37 (data), 3.46 (BB)
MSE = 18.48 (data), 19.00 (BB)
R2 = 0.94 (data), 0.94 (BB)

CReEPy 

In [22]:
crash = CRASH(predictor, train, max_depth=3, patience=1, readability_tradeoff=.75, algorithm=CRASH.Algorithm.CREAM)
crash.search()
(_, _, depth, threshold) = crash.get_best()[0]

creepy = Extractor.creepy(predictor, depth=depth, error_threshold=threshold, output=Target.REGRESSION,
                          clustering=Clustering.cream)
theory_from_creepy = creepy.extract(train)
scores, completeness = get_scores(creepy, test, predictor)
print(f'CReEPy performance ({creepy.n_rules} rules with {completeness * 100:.2f}% coverage):')
print_scores(scores)
print('\nCReEPy extracted rules:\n\n' + pretty_theory(theory_from_creepy))

Algorithm.CREAM. Depth: 1. Threshold = 0.00. MAE = 10.29, 2 rules
Algorithm.CREAM. Depth: 1. Threshold = 0.00. MAE = 8.46, 2 rules
Algorithm.CREAM. Depth: 1. Threshold = 0.00. MAE = 10.29, 2 rules

Algorithm.CREAM. Depth: 2. Threshold = 0.00. MAE = 6.46, 4 rules
Algorithm.CREAM. Depth: 2. Threshold = 0.00. MAE = 6.10, 4 rules
Algorithm.CREAM. Depth: 2. Threshold = 0.00. MAE = 7.75, 4 rules

Algorithm.CREAM. Depth: 3. Threshold = 0.00. 

KeyboardInterrupt: 

In [ ]:
it = Extractor.iter(predictor, min_update=1.0 / 10, n_points=1, max_iterations=600,
                    min_examples=100, threshold=5)
theory_from_iter = it.extract(train)
scores, completeness = get_scores(it, test, predictor)
print(f'ITER performance ({it.n_rules} rules with {completeness * 100:.2f}% coverage):')
print_scores(scores)
print('\nITER extracted rules:\n\n' + pretty_theory(theory_from_iter))

In [11]:
pedro = PEDRO(predictor, train, max_mae_increase=1.2, min_rule_decrease=0.9, readability_tradeoff=0.1,
              max_depth=1, patience=1, algorithm=PEDRO.Algorithm.GRIDEX, objective=Objective.MODEL)
pedro.search()
(_, _, threshold, grid) = pedro.get_best()[0]

gridEx = Extractor.gridex(predictor, grid, threshold=threshold)
theory_from_gridEx = gridEx.extract(train)
scores, completeness = get_scores(gridEx, test, predictor)
print(f'GridEx performance ({gridEx.n_rules} rules with {completeness * 100:.2f}% coverage):')
print_scores(scores)
print('\nGridEx extracted rules:\n\n' + pretty_theory(theory_from_gridEx))

Algorithm.GRIDEX. Grid (1). Fixed (2). Threshold = 0.00. MAE = 6.45, 15 rules
Algorithm.GRIDEX. Grid (1). Fixed (2). Threshold = 0.00. MAE = 6.45, 30 rules

Algorithm.GRIDEX. Grid (1). Fixed (3). Threshold = 0.00. MAE = 6.45, 86 rules
Algorithm.GRIDEX. Grid (1). Fixed (3). Threshold = 0.00. MAE = 6.45, 142 rules

Algorithm.GRIDEX. Grid (1). Adaptive ([(0.99, 2)]). Threshold = 0.00. MAE = 6.45, 144 rules
Algorithm.GRIDEX. Grid (1). Adaptive ([(0.99, 2)]). Threshold = 0.00. MAE = 6.45, 146 rules

Algorithm.GRIDEX. Grid (1). Adaptive ([(0.3, 2)]). Threshold = 0.00. MAE = 6.45, 150 rules
Algorithm.GRIDEX. Grid (1). Adaptive ([(0.3, 2)]). Threshold = 0.00. MAE = 6.45, 154 rules

Algorithm.GRIDEX. Grid (1). Adaptive ([(0.99, 3)]). Threshold = 0.00. MAE = 6.45, 157 rules
Algorithm.GRIDEX. Grid (1). Adaptive ([(0.99, 3)]). Threshold = 0.00. MAE = 6.45, 160 rules

Algorithm.GRIDEX. Grid (1). Adaptive ([(0.3, 3)]). Threshold = 0.00. MAE = 6.45, 169 rules
Algorithm.GRIDEX. Grid (1). Adaptive ([(0

In [12]:
#pedro = PEDRO(predictor, train, max_mae_increase=1.2, min_rule_decrease=0.9, readability_tradeoff=0.1,
#              max_depth=2, patience=1, algorithm=PEDRO.Algorithm.GRIDREX, objective=Objective.MODEL)
#pedro.search()
(_, _, threshold, grid) = pedro.get_best()[0]

gridREx = Extractor.gridrex(predictor, grid, threshold=threshold)
theory_from_gridREx = gridREx.extract(train)
scores, completeness = get_scores(gridREx, test, predictor)
print(f'GridREx performance ({gridREx.n_rules} rules with {completeness * 100:.2f}% coverage):')
print_scores(scores)
print('GridREx extracted rules:\n\n' + pretty_theory(theory_from_gridREx))

**********************
Best Algorithm.GRIDEX
**********************
MAE = 6.45, 15 rules
Threshold = 0.00
Iterations = 1
Strategy = Fixed (2)

**********************
Best   MAE  
**********************
MAE = 6.45, 256 rules
Threshold = 0.00
Iterations = 1
Strategy = Adaptive ([(0.33, 2), (0.67, 3)])

**********************
Best N rules
**********************
MAE = 6.45, 15 rules
Threshold = 0.00
Iterations = 1
Strategy = Fixed (2)

GridREx performance (292 rules with 99.90% coverage):
MAE = 6.50 (data), 6.49 (BB)
MSE = 66.41 (data), 65.91 (BB)
R2 = 0.77 (data), 0.77 (BB)
GridREx extracted rules:

'PE'(AP, AT, RH, V, 473.96) :-
    AT in [2.33, 19.05], V in [25.35, 53.46], AP in [992.88, 1013.07], RH in [25.55, 62.85].
'PE'(AP, AT, RH, V, 468.97) :-
    AT in [2.33, 19.05], V in [25.35, 53.46], AP in [992.88, 1013.07], RH in [62.85, 100.15].
'PE'(AP, AT, RH, V, 471.39) :-
    AT in [2.33, 19.05], V in [25.35, 53.46], AP in [1013.07, 1033.25], RH in [25.55, 62.85].
'PE'(AP, AT, RH, V, 47

In [13]:
cart = Extractor.cart(predictor, max_depth=5, max_leaves=6, simplify=True)
theory_from_cart = cart.extract(train)
scores, completeness = get_scores(cart, test, predictor)
print(f'CART performance ({cart.n_rules} rules with {completeness * 100:.2f}% coverage):')
print_scores(scores)
print('CART extracted rules:\n\n' + pretty_theory(theory_from_cart))

CART performance (6 rules with 100.00% coverage):
MAE = 4.46 (data), 4.49 (BB)
MSE = 32.55 (data), 32.53 (BB)
R2 = 0.89 (data), 0.89 (BB)
CART extracted rules:

'PE'(AP, AT, RH, V, 479.15) :-
    AT =< 18.25, AT =< 11.90.
'PE'(AP, AT, RH, V, 435.66) :-
    AT > 18.25, V > 66.20.
'PE'(AP, AT, RH, V, 451.33) :-
    AT > 18.25, V =< 66.20, AT =< 22.89.
'PE'(AP, AT, RH, V, 443.00) :-
    AT > 18.25, V =< 66.20, AT > 22.89.
'PE'(AP, AT, RH, V, 467.45) :-
    AT > 11.90, AT =< 15.64.
'PE'(AP, AT, RH, V, 459.70) :-
    AT > 15.64.
